In [5]:
!ls

db.db         premium.ipynb


In [1]:
from dateutil.relativedelta import relativedelta

In [2]:
import numpy as np

In [3]:
DATES_COLS = ['date_start', 'date_end']

In [6]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('../db/db.db')

df = pd.read_sql_query("SELECT * FROM premium_payments", cnx, parse_dates=DATES_COLS)

In [7]:
df.head()

,member_id,date_start,date_end,type
0,1,2018-10-31,2018-11-30,1
1,1,2018-12-01,2019-01-01,1
2,2,2019-01-18,2019-02-17,3
3,2,2019-02-19,2019-03-22,3
4,2,2019-03-23,2019-04-23,3


In [8]:
df.shape

(24999, 4)

In [9]:
df.dtypes

member_id              int64
date_start    datetime64[ns]
date_end      datetime64[ns]
type                   int64
dtype: object

In [ ]:
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

In [ ]:
df

In [10]:
df.member_id.value_counts()

5000    4
9148    4
1553    4
7393    4
4740    4
       ..
3149    1
3180    1
8262    1
3194    1
9999    1
Name: member_id, Length: 9999, dtype: int64

In [ ]:
# id 1553 get differents type of premium

In [ ]:
df[df['member_id'] == 1553]

In [ ]:
sample = df[df['member_id'] == 1553]

In [ ]:
sample

In [ ]:
sample['test'] = sample.groupby(['member_id', 'type'])['date_end'].shift(1)

In [ ]:
sample

In [ ]:
sample['date_start'].min()

In [ ]:
#df.merge(df.groupby(['C', 'D']).apply(lambda row: row['B'] - row['A']).sum(level=[0,1]).reset_index())

In [ ]:
sample.groupby(['member_id', 'type']).apply(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
sample.groupby(['member_id', 'type']).transform(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
result = sample.groupby(['member_id', 'type']).apply(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
result

In [ ]:
sample['relativedelta'] = 

In [ ]:
sample['transaction_class'] = np.where(sample['date_start']+pd.Timedelta(3, unit='D')<sample['test'], 'other_transaction', 'same transaction')

In [ ]:
sample

In [ ]:
sample.groupby(['member_id', 'type', 'transaction_class']).agg({
    'date_start': min,
    'date_end': max
})

In [ ]:
sample['test_bis'] = sample['date_start'] - sample['test']

In [ ]:
sample

In [ ]:
sample['test_bis'] + pd.Timedelta(3, unit='D') 

In [12]:
experiment = df.copy()

In [ ]:
# get number of members
df.member_id.nunique()

In [11]:
def _group_transaction(df):
    df['end_date_alg'] = df.groupby(['member_id', 'type'])['date_end'].shift(1)
    return np.where(df['date_start']+pd.Timedelta(3, unit='D')<df['end_date_alg'], 'other_transaction', 'same transaction')

In [ ]:
df['date_start']+pd.Timedelta(3, unit='D')

In [13]:
experiment['transaction_class'] = _group_transaction(experiment)

In [14]:
experiment

,member_id,date_start,date_end,type,end_date_alg,transaction_class
0,1,2018-10-31,2018-11-30,1,NaT,same transaction
1,1,2018-12-01,2019-01-01,1,2018-11-30,same transaction
2,2,2019-01-18,2019-02-17,3,NaT,same transaction
3,2,2019-02-19,2019-03-22,3,2019-02-17,same transaction
4,2,2019-03-23,2019-04-23,3,2019-03-22,same transaction
...,...,...,...,...,...,...
24994,9998,2019-06-05,2019-07-05,2,NaT,same transaction
24995,9998,2019-09-03,2019-10-04,1,NaT,same transaction
24996,9998,2019-10-04,2019-11-04,1,2019-10-04,same transaction
24997,9998,2019-11-06,2019-12-07,1,2019-11-04,same transaction


In [15]:
experiment.transaction_class.value_counts()

same transaction    24999
Name: transaction_class, dtype: int64

In [16]:
result = experiment.groupby(['member_id', 'type', 'transaction_class']).agg({
    'date_start': min,
    'date_end': max
})

In [17]:
result

date_start   date_end
member_id type transaction_class                      
1         1    same transaction  2018-10-31 2019-01-01
2         3    same transaction  2019-01-18 2019-04-23
3         3    same transaction  2019-03-30 2019-07-29
4         2    same transaction  2019-11-29 2020-03-16
5         3    same transaction  2019-11-28 2020-01-29
...                                     ...        ...
9997      1    same transaction  2019-01-07 2019-06-09
          2    same transaction  2019-02-08 2019-03-11
9998      1    same transaction  2019-09-03 2019-12-07
          2    same transaction  2019-06-05 2019-07-05
9999      1    same transaction  2019-03-14 2019-04-13

[12725 rows x 2 columns]

In [ ]:
result.columns

In [ ]:
result.index

In [ ]:
result.reset_index(level='transaction_class').drop('transaction_class', axis=1)

In [18]:
final_result = result.reset_index().drop('transaction_class', axis=1)

In [19]:
final_result

,member_id,type,date_start,date_end
0,1,1,2018-10-31,2019-01-01
1,2,3,2019-01-18,2019-04-23
2,3,3,2019-03-30,2019-07-29
3,4,2,2019-11-29,2020-03-16
4,5,3,2019-11-28,2020-01-29
...,...,...,...,...
12720,9997,1,2019-01-07,2019-06-09
12721,9997,2,2019-02-08,2019-03-11
12722,9998,1,2019-09-03,2019-12-07
12723,9998,2,2019-06-05,2019-07-05


In [ ]:
final_result.member_id.value_counts()

In [ ]:
final_result[final_result['member_id']==9637]